### ETL S3 servidores públicos

In [42]:
### importar bibliotecas de python para realizar la migracion
import json
from addict import Dict
import glob
from pprint import pprint

In [43]:
### ruta de los archivos json
ruta_datos_entrada = '../pruebas/datos_entrada'
ruta_datos_salida = '../pruebas/datos_salida'

# Apuntando a un solo archivo de entrada para pruebas en s3s
archivo_entrada_s3s = '/s3s/CHIAPAS/data-0000000001.json'

#nameFiles = glob.glob('*.json')

In [44]:
### varibables globales
array_faltas_servidores_graves = []
array_faltas_servidores_no_graves = []


In [45]:
### diccionario de faltas graves
tipo_de_faltas_graves = [
  "ABUSO_FUNCIONES", 
"COHECHO", 
"PECULADO", 
"DESVIO_RECURSOS_PUBLICOS", 
"UTILIZACION_INDEBIDA_INFORMACION", 
"CONFLICTO_INTERES", 
"CONTRATACION_INDEBIDA", 
"ENRIQUECIMIENTO_OCULTO", 
"TRAFICO_INFLUENCIAS", 
"SIMULACION_ACTO_JURIDICO", 
"ENCUBRIMIENTO", 
"DESACATO", 
"NEPOTISMO", 
"OBSTRUCCION"
]

In [46]:
### diccionario de faltas no graves
tipo_de_faltas_no_grave = ["CUMPLIR_FUNCIONES", 
"DENUNCIAR_FALTAS", 
"ATENDER_INSTRUCCIONES", 
"DECLARACIONES", 
"CUSTODIAR_DOCUMENTACION", 
"SUPERVISAR_ART_49_LGRA", 
"RENDIR_CUENTAS", 
"COLABORAR_PROCEDIMIENTOS", 
"CAUSAR_DANOS"
]

In [47]:
### Se carga los datos de los archivos JSON en un array de objetos Dict
def load_json_files(file_pattern):
    all_data = []
    for filename in glob.glob(file_pattern):
        with open(filename, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
            # Convertir cada objeto JSON a un objeto Dict de addict
            all_data.extend([Dict(item) for item in json_data])
    return all_data

In [48]:
### Cargar todos los archivos JSON que se ubiquen en la ruta especificada
data = load_json_files(ruta_datos_entrada + archivo_entrada_s3s)

In [64]:
### old
""" def extract_faltas_element(element_data):
    processed_element = {
        'fecha': element_data.fechaCaptura,
        'expediente': element_data.expediente,
        'datosGenerales': {
            'nombre': element_data.servidorPublicoSancionado.nombres,
            'primerApellido': element_data.servidorPublicoSancionado.primerApellido,
            'segundoApellido': element_data.servidorPublicoSancionado.segundoApellido,
            if element_data.servidorPublicoSancionado.curp is not None:
                'curp': element_data.servidorPublicoSancionado.curp,
            else:
                'curp': None,
            'rfc': element_data.servidorPublicoSancionado.rfc,
            'sexo': element_data.servidorPublicoSancionado.sexo,
        },
    }
    return processed_element
 """

In [67]:
def extract_faltas_element(element_data):
    # Función auxiliar para acceder de forma segura a atributos anidados
    def safe_get(obj, *attrs):
        for attr in attrs:
            if not hasattr(obj, attr):
                return None
            obj = getattr(obj, attr)
        return obj

    # Construir el diccionario de datos generales
    datos_generales = {
        'nombre': safe_get(element_data, 'servidorPublicoSancionado', 'nombres'),
        'primerApellido': safe_get(element_data, 'servidorPublicoSancionado', 'primerApellido'),
        'segundoApellido': safe_get(element_data, 'servidorPublicoSancionado', 'segundoApellido'),
        'rfc': safe_get(element_data, 'servidorPublicoSancionado', 'rfc'),
        'sexo': safe_get(element_data, 'servidorPublicoSancionado', 'sexo'),
    }

    # Añadir CURP solo si existe
    curp = safe_get(element_data, 'servidorPublicoSancionado', 'curp')
    if curp is not None:
        datos_generales['curp'] = curp

    # Construir el diccionario procesado
    processed_element = {
        'fecha': safe_get(element_data, 'fechaCaptura'),
        'expediente': safe_get(element_data, 'expediente'),
        'datosGenerales': datos_generales
    }

    return processed_element

# Ejemplo de uso:
# element_data = ... # Tu objeto con los datos
# resultado = extract_faltas_element(element_data)
# print(resultado)

In [68]:
### Extraer datos de un solo elemento del array
element_data = data[0]
##pprint(element_data)
processed_element = extract_faltas_element(element_data)


In [69]:
pprint(processed_element)

{'datosGenerales': {'curp': {},
                    'nombre': 'ANDRES',
                    'primerApellido': 'REYES',
                    'rfc': {},
                    'segundoApellido': 'ENRIQUEZ',
                    'sexo': {}},
 'expediente': 'CA/012/2022',
 'fecha': '2022-05-06T00:00:00Z'}


In [70]:
pprint(data[0].tipoFalta.valor)

'OTRO'


In [52]:
pprint(type(data))
pprint(len(data))
pprint(len(data[0].keys()))
pprint(data[0].keys())
pprint(data[0])
pprint(data[0].id)

<class 'list'>
27
12
dict_keys(['id', 'fechaCaptura', 'expediente', 'institucionDependencia', 'servidorPublicoSancionado', 'autoridadSancionadora', 'tipoFalta', 'tipoSancion', 'causaMotivoHechos', 'resolucion', 'multa', 'inhabilitacion'])
{'autoridadSancionadora': 'DATOS NO PROPORCIONADOS',
 'causaMotivoHechos': 'DATOS NO PROPORCIONADOS',
 'expediente': 'CA/012/2022',
 'fechaCaptura': '2022-05-06T00:00:00Z',
 'id': 'b411f02b-5ea7-489e-93e5-9eca24d090f6',
 'inhabilitacion': {'fechaFinal': '2022-05-15',
                    'fechaInicial': '2022-02-15',
                    'plazo': '3 MESES'},
 'institucionDependencia': {'clave': '78',
                            'nombre': 'Fiscalia General del Estado',
                            'siglas': 'FGE'},
 'multa': {'moneda': {'clave': 'MXN', 'valor': 'PESO MEXICANO'}, 'monto': 0},
 'resolucion': {'fechaResolucion': '2022-01-12',
                'url': 'DATOS NO PROPORCIONADOS'},
 'servidorPublicoSancionado': {'nivel': 'DATOS NO PROPORCIONADOS',

In [53]:
pprint(processed_element)

{'faltas_servidores_graves': [],
 'faltas_servidores_no_graves': [],
 'id': 'b411f02b-5ea7-489e-93e5-9eca24d090f6',
 'nombre': {}}
